# Analýza záznamů robotů
budeme pracovat s daty robotu z nasich databazi. Podivame se na nastroje `BigData`

## Import knihoven a konstanty

In [ ]:
import os
import datetime
import pandas
import sqlalchemy as sql

In [ ]:
__file__ = __vsc_ipynb_file__  
SCRIPT_FOLDER = os.path.dirname(__file__)
PROJECT_FOLDER = os.path.dirname(SCRIPT_FOLDER)
DATA_FOLDER = os.path.join(PROJECT_FOLDER,'data')
WORK_FOLDER = os.path.join(DATA_FOLDER,'work')

COL_TIME = 'cas_robota'
COL_ROBOT = 'robot'
COL_DAY = 'day'
COL_MINUTE = 'minute'

PRQT = os.path.join(WORK_FOLDER,'roboti.prqt')
CSV = os.path.join(WORK_FOLDER,'roboti.csv')
XLS = os.path.join(WORK_FOLDER,'roboti.xlsx')
PRQT_PART = os.path.join(WORK_FOLDER,'roboti.part')

## Přístupové údaje do databází

In [ ]:
DBSURL = "https://development.techniarch.com/pcsda/robot.php?dbs=psycopg2"
DBUSER = 'robot'
DBPASS = 'Kurz-DAPYT_ST45'

## Získání seznamu databází

## Přečtení dat robotů
z SQL DB a vytvoření společného `DataFrame`

## Drobné úpravy v robotech

## Uložení DF na lokální disk
### CSV

### Excel

### PARQUET
Soubory `parquet`
- binární soubor(y)
- soubory jsou komprimované
- umožňují indexování a partitioning
- není možné soubor měnit, jen přepsat /POZOR: na adresářové struktury/
- připraveno pro velké objemy data - `BigData`

## Čtení celého souboru
### CSV

### PARQUET

## Čtení části dat
(potřebujeme data jen jednoho robota)

### Klasický přístup
přečteme vše a filtrujem

#### Parquet

### Využití PARQUET partitioning

#### 1. uložíme data rozdělená na části `partition`

#### 2. čteme data s využitím filtru